In [1]:
# Import required libraries
from jupyter_dash import JupyterDash
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.drop(spacex_df.columns[0],axis=1, inplace=True)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

#print(max_payload,min_payload)
site_list=spacex_df['Launch Site'].unique().tolist()
site_list.append('ALL')

In [2]:
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
df1=spacex_df.groupby('class').get_group(0)
df11=pd.DataFrame(df1.groupby('Launch Site')['class'].count())
df11.columns=['Failure']
df11=df11.sort_index()
df11.reset_index(inplace=True)
df2=spacex_df.groupby('class').get_group(1)
df22=pd.DataFrame(df2.groupby('Launch Site')['class'].count())
df22.columns=['Success']
df22=df22.sort_index()
df22.reset_index(inplace=True)
df=pd.concat([df11,df22['Success']],axis=1)
df.head()

,Launch Site,Failure,Success
0,CCAFS LC-40,19,7
1,CCAFS SLC-40,4,3
2,KSC LC-39A,3,10
3,VAFB SLC-4E,6,4


In [4]:
# Create a dash application
#app = dash.Dash(__name__)
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                html.Div(
                                        [dcc.Dropdown(id='l_site',
                                                      # Update dropdown values using list comphrehension
                                                      options=[{'label': i, 'value': i} for i in site_list],
                                                      value="ALL",
  #                                                    placeholder="Select a Site",
                                                      style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                        ], style={'display': 'flex'}
                                ),
                                   html.Br(),
                                   html.Br(),
                                   html.Br(),                                
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(
                                    [dcc.RangeSlider(id='payload-slider',min=min_payload,max=max_payload, value=[min_payload,max_payload], tooltip={'always_visible':True}),    
                                ]),
                                
                                html.Div([
                                        html.Div([ ], id='plot2')
                                ],style={'display': 'flex'}),  
                                
                                 html.Div([
                                        html.Div([ ], id='plot1')
                                ],style={'display': 'flex'}),                                
#                                html.Div(dcc.Graph(id='success-pie-chart')),
#                                html.Br(),

#                                html.P("Payload range (Kg):"),
#                                # TASK 3: Add a slider to select payload range
#                                #dcc.RangeSlider(id='payload-slider',...)

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
#                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])





# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback([ Output(component_id='plot1', component_property='children'),
                Output(component_id='plot2', component_property='children')
              ],
              [ Input(component_id='l_site', component_property='value'),
                Input(component_id='payload-slider', component_property='value')
              ],
               [
                State("plot1", 'children'), State("plot2", "children")
               ]
            )



def get_graph(site,value1,value2,value3):
    rr=str(value1)
#    print(site,value1,value2,value3)
#    print(value1[0],value1[1])


    if site == 'ALL':
        pie_fig = px.pie(df, values='Success', names='Launch Site', title='Total Successful Launches') 
    else:
        df2=df.T
        df2.columns=df2.iloc[0]
        df2=df2[1:]
#        pie_fig = px.bar(df2,x='Launch Site',y=['Success','Failure'], title='Total Successful/Failure Launches')
        pie_fig = px.pie(df2, values=site, names=df2.index, title=site)  

    df3=spacex_df[spacex_df['Payload Mass (kg)'].between(value1[0],value1[1])]


    sct_fig = px.scatter(df3, x='Payload Mass (kg)',y='class', color='class', title='Pay Load Vs Success ')
    sct_fig.update_traces(marker_size=12)
    
    
    return [dcc.Graph(figure=pie_fig),
            dcc.Graph(figure=sct_fig)
           ]

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output


# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline')
#    app.run_server()